# Welcome to Zarr and Dask for large-scale imaging workshop
## Fernando Cervantes
### Systems Analyst in JAX's Research IT
### email: fernando.cervantes@jax.org

## Outcomes for today's session:
- Learn to use Dask library with Zarr image data
- Implement and apply image analysis pipelines with Dask
- Save image analysis outputs as Zarr files


---
# Overview of the Dask package

Dask is lazy!

![image](https://docs.dask.org/en/stable/_images/dask-array.svg)

# 1. Manipulate Dask arrays

## 1.1 Create Dask arrays

In [ ]:
import dask
import dask.array as da
import numpy as np

In [ ]:
d1 = da.zeros((10, 10), chunks=(5, 5), dtype=np.int16)

In [ ]:
d1

In [ ]:
d1[:5, :5] = 1

In [ ]:
d1

---
## 1.3 Eexecute the computation graph

In [ ]:
d1[:].compute()

In [ ]:
d1 = da.zeros((10, 10), chunks=(5, 5))

In [ ]:
d1[3:8, :5] = 2

In [ ]:
d1.compute()

In [ ]:
d1 = d1 + 1

In [ ]:
d2 = da.ones((10, 10), chunks=(3, 3))

In [ ]:
d3 = d1 + d2

In [ ]:
d3

In [ ]:
d3.chunks

---
## 1.4 Rechunk Dask arrays

In [ ]:
d3 = d3.rechunk((5, 5))

In [ ]:
d3

In [ ]:
d3 = d1 + d2.rechunk(d1.chunks)

In [ ]:
d3

In [ ]:
d3_sum = np.sum(d3)

In [ ]:
d3_sum

In [ ]:
d3_sum.compute()

In [ ]:
d3_cos = np.cos(d3)

In [ ]:
d3_cos

In [ ]:
d3_cos.compute()

---
## 1.5 Persist vs Compute

In [ ]:
d3 = d1 + d2.rechunk((5, 5))

In [ ]:
d3 = d3.persist()

In [ ]:
d3 = d3 + 1

---
# 2. Open Zarr files with Dask

In [ ]:
import zarr
import tifffile
import matplotlib.pyplot as plt

In [ ]:
z_grp = tifffile.imread("CMU-1.svs", aszarr=True)
da_arr = da.from_zarr(z_grp, component="0")

In [ ]:
da_arr

In [ ]:
da_arr = da_arr.rechunk((512, 512, 3))

In [ ]:
da_arr

---
# 3. Perform image analysis on Dask arrays

In [ ]:
import skimage

In [ ]:
da_sel = da_arr[10000:10000 + 2048, 10000:10000 + 2048]

In [ ]:
da_sel

In [ ]:
da_sel.compute()

ℹ Dask arrays already work with `skimage` functions without having to execute `.compute()`

In [ ]:
arr_gray = skimage.color.rgb2gray(da_sel)

In [ ]:
type(arr_gray)

In [ ]:
arr_nuclei = arr_gray < 0.25

In [ ]:
plt.imshow(arr_nuclei, cmap="gray")